In [ ]:
!mkdir -p /root/.kaggle/

import json
token = {"username":"abdelino17","key":"4d98f713923ea94844c503f0fa7e01f7"}
with open('/root/.kaggle/kaggle.json', 'w') as file:
    json.dump(token, file)
    
!cat /root/.kaggle/kaggle.json

!chmod 600 /root/.kaggle/kaggle.json

!kaggle datasets download -d vbookshelf/v2-plant-seedlings-dataset -p /content

!mkdir dataset && unzip -d dataset v2-plant-seedlings-dataset.zip

from google.colab import drive

drive.mount('/content/gdrive')

In [1]:
!rm -rf data2 data

In [3]:
RANDOM_SEED = 42
DATASET_DIR = "../dataset"
TRAIN_PERCENTAGE = 0.75
AVOID_CLASS = "Black-grass"

In [4]:
import cv2
from glob import glob
import numpy as np
from matplotlib import pyplot as plt
import math
import pandas as pd
import os
import random
import shutil
import tensorflow as tf

np.random.seed(RANDOM_SEED)
tf.set_random_seed(RANDOM_SEED)
random.seed(RANDOM_SEED)

In [5]:
total = 0
for dir in os.listdir(DATASET_DIR):
    label = dir
    num_files = len(os.listdir(os.path.join(DATASET_DIR, label)))
    print(label, num_files)
    total += num_files
print("----------------------")
print("Total number of images:", total)

Black-grass 309
Charlock 452
Cleavers 335
Common Chickweed 713
Common wheat 253
Fat Hen 538
Loose Silky-bent 762
Maize 257
Scentless Mayweed 607
Shepherd's Purse 274
Small-flowered Cranesbill 576
Sugar beet 463
----------------------
Total number of images: 5539


In [6]:
!rm -rf data

data_dir = 'data'
train_dir = os.path.join(data_dir, 'train')
test_dir = os.path.join(data_dir, 'test')

os.mkdir(data_dir)
os.mkdir(train_dir)
os.mkdir(test_dir)

for dir in os.listdir(DATASET_DIR):
    path = os.path.join(DATASET_DIR, str(dir))
    files = os.listdir(path)
    num_files = len(files)
    num_train_files = int(num_files * TRAIN_PERCENTAGE) 
    
    train_files = random.sample(files, num_train_files)
    
    test_files = [file for file in files if file not in train_files]
    
    i=0
    j=0
    
    for file in train_files:  
        if i==0:
            os.mkdir(os.path.join(train_dir, dir))
        src = os.path.join(path, file)
        dst = os.path.join(train_dir, dir, file)
        shutil.copy(src, dst)
        i+=1
    
    for file in test_files:  
        if j==0:
            os.mkdir(os.path.join(test_dir, dir))
        src = os.path.join(path, file)
        new_file = str(dir) + "_" + file
        dst = os.path.join(test_dir, dir, new_file)
        shutil.copy(src, dst)
        j+=1

In [7]:
print("All classes")
total = 0
for dir in os.listdir(train_dir):
    label = dir
    num_files = len(os.listdir(os.path.join(train_dir, label)))
    print(label, num_files)
    total += num_files
print("----------------------")
print("Total number of images in the train:", total)
print("--------------------------------------------------------------------------")
total = 0
for dir in os.listdir(test_dir):
    label = dir
    num_files = len(os.listdir(os.path.join(test_dir, label)))
    print(label, num_files)
    total += num_files
print("----------------------")
print("Total number of images in the test:", total)

All classes
Black-grass 231
Charlock 339
Cleavers 251
Common Chickweed 534
Common wheat 189
Fat Hen 403
Loose Silky-bent 571
Maize 192
Scentless Mayweed 455
Shepherd's Purse 205
Small-flowered Cranesbill 432
Sugar beet 347
----------------------
Total number of images in the train: 4149
--------------------------------------------------------------------------
Black-grass 78
Charlock 113
Cleavers 84
Common Chickweed 179
Common wheat 64
Fat Hen 135
Loose Silky-bent 191
Maize 65
Scentless Mayweed 152
Shepherd's Purse 69
Small-flowered Cranesbill 144
Sugar beet 116
----------------------
Total number of images in the test: 1390


In [8]:
from keras.preprocessing.image import ImageDataGenerator,array_to_img, img_to_array, load_img
import sys
from PIL import Image
sys.modules['Image'] = Image 

Using TensorFlow backend.


In [9]:
datagen = ImageDataGenerator(
        brightness_range=[0.1, 2],
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=True,  # apply ZCA whitening
        rotation_range=360,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range=0.2, # Randomly zoom image 
        width_shift_range=0.3,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.3,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        fill_mode='nearest',
        vertical_flip=True)  # randomly flip images

C:\Users\Abdel\Anaconda3\envs\tf-gpu\lib\site-packages\keras_preprocessing\image\image_data_generator.py:334: UserWarning: This ImageDataGenerator specifies `zca_whitening`, which overrides setting of `featurewise_center`.
  warnings.warn('This ImageDataGenerator specifies '


In [10]:
number_samples_per_class = 600

!rm -rf data/train_augmented
src_dir = train_dir
dst_dir = os.path.join(data_dir, 'train_augmented')
os.mkdir(dst_dir)

In [11]:
!rm -rf data/train_augmented/*

j=0
for dir in os.listdir(src_dir):
    path = os.path.join(src_dir, str(dir))
    dst_class = os.path.join(dst_dir, str(dir))
    files = os.listdir(path)
    count = len(files)
    print("Directory ", j)
    ratio = math.floor(number_samples_per_class / count) - 1
    for file in files:  
        if not os.path.exists(dst_class):
            os.mkdir(dst_class)
        img = load_img(os.path.join(path, file))
        new_img = img_to_array(img)
        new_img = new_img.reshape((1, ) + new_img.shape)
        i = 0
        for batch in datagen.flow(new_img, batch_size=1, save_to_dir=dst_class, save_format='jpg'):
            i+=1
            if i > ratio:
                break
    dst_files = os.listdir(dst_class)
    missing = number_samples_per_class - len(dst_files)
    while missing > 0:
        missing_samples = random.sample(files, missing)
        print("Missing elts: ", missing)
        for file in missing_samples:
            img = load_img(os.path.join(path, file))
            new_img = img_to_array(img)
            new_img = new_img.reshape((1, ) + new_img.shape)
            gen =  datagen.flow(new_img, batch_size=1, save_to_dir=dst_class, save_format='jpg')
            next(gen)
        dst_files = os.listdir(dst_class)
        missing = number_samples_per_class - len(dst_files)
        j+=1
    print("-----------------------------------------------")

Directory  0


C:\Users\Abdel\Anaconda3\envs\tf-gpu\lib\site-packages\keras_preprocessing\image\image_data_generator.py:699: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
C:\Users\Abdel\Anaconda3\envs\tf-gpu\lib\site-packages\keras_preprocessing\image\image_data_generator.py:718: UserWarning: This ImageDataGenerator specifies `zca_whitening`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


Missing elts:  147
Missing elts:  9
-----------------------------------------------
Directory  2
Missing elts:  263
Missing elts:  14
Missing elts:  1
-----------------------------------------------
Directory  5
Missing elts:  106
Missing elts:  3
-----------------------------------------------
Directory  7
Missing elts:  77
Missing elts:  4
-----------------------------------------------
Directory  9
Missing elts:  52
Missing elts:  6
Missing elts:  1
-----------------------------------------------
Directory  12
Missing elts:  205
Missing elts:  13
-----------------------------------------------
Directory  14
Missing elts:  44
Missing elts:  3
-----------------------------------------------
Directory  16
Missing elts:  41
-----------------------------------------------
Directory  17
Missing elts:  157
Missing elts:  11
Missing elts:  1
-----------------------------------------------
Directory  20
Missing elts:  197
Missing elts:  6
-----------------------------------------------
Direc

In [12]:
print("All classes (with Data augmentation) \n")

total = 0
for dir in os.listdir(dst_dir):
    label = dir
    num_files = len(os.listdir(os.path.join(dst_dir, label)))
    print(label, num_files)
    total += num_files
print("----------------------")
print("Total number of images in the train dir with data augmentation:", total)
print("--------------------------------------------------------------------------")

All classes (with Data augmentation) 

Black-grass 600
Charlock 600
Cleavers 600
Common Chickweed 600
Common wheat 600
Fat Hen 600
Loose Silky-bent 600
Maize 600
Scentless Mayweed 600
Shepherd's Purse 600
Small-flowered Cranesbill 600
Sugar beet 600
----------------------
Total number of images in the train dir with data augmentation: 7200
--------------------------------------------------------------------------


In [ ]:
os.listdir(dst_dir)

In [ ]:
!cp /content/gdrive/My\ Drive/Colab\ Notebooks/final-model.pth .

In [ ]:
plot_model(model, to_file='model.png')
SVG(model_to_dot(model).create(prog='dot', format='svg'))

In [13]:
import os
import cv2

import imageio
import skimage
import skimage.io
import skimage.transform

from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
import itertools
import shutil
import matplotlib.pyplot as plt
%matplotlib inline

ModuleNotFoundError: No module named 'imageio'

In [14]:
folder_dir = './data/test/'
folder_list = os.listdir(folder_dir)
folder_list

['Black-grass',
 'Charlock',
 'Cleavers',
 'Common Chickweed',
 'Common wheat',
 'Fat Hen',
 'Loose Silky-bent',
 'Maize',
 'Scentless Mayweed',
 "Shepherd's Purse",
 'Small-flowered Cranesbill',
 'Sugar beet']

In [15]:
!rm -rf data/test_cleaned
new_test_dir = os.path.join("data", "test_cleaned")
os.mkdir(new_test_dir)

for folder in folder_list:
    
    # create a path to the folder
    path = folder_dir + str(folder)

    # create a list of all files in the folder
    file_list = os.listdir(path)

    # move the 0 images to all_images_dir
    for fname in file_list:

        # source path to image
        src = os.path.join(path, fname)
        
        # Change the file name because many images have the same file name.
        # Add the folder name to the existing file name.
        
        # destination path to image
        dst = os.path.join(new_test_dir, fname)
        # copy the image from the source to the destination
        shutil.copyfile(src, dst)

In [ ]:
! rm -rf data/train_augmented/models

In [ ]:
!rm -rf data/train_segmented
seg_test_dir = os.path.join("data", "train_segmented")
os.mkdir(seg_test_dir)
train_augmented = os.path.join("data", "train_augmented")
train_augmented_list = os.listdir(train_augmented)

for folder in train_augmented_list:
    
    # create a path to the folder
    path = os.path.join(train_augmented, str(folder))
    dst_path = os.path.join(seg_test_dir, str(folder))
    # create a list of all files in the folder
    file_list = os.listdir(path)

    # move the 0 images to all_images_dir
    for fname in file_list:
      
        if not os.path.exists(dst_path):
            os.mkdir(dst_path)

        # source path to image
        src = os.path.join(path, fname)
        img = cv2.imread(src)
        blurImg = cv2.GaussianBlur(img, (5, 5), 0)   
    
        # Convert to HSV image
        hsvImg = cv2.cvtColor(blurImg, cv2.COLOR_BGR2HSV)  

        # Create mask (parameters - green color range)
        lower_green = (25, 40, 50)
        upper_green = (75, 255, 255)
        mask = cv2.inRange(hsvImg, lower_green, upper_green)  
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (11, 11))
        mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)

        # Create bool mask
        bMask = mask > 0  

        # Apply the mask
        clear = np.zeros_like(img, np.uint8)  # Create empty image
        clear[bMask] = img[bMask]
        
        # destination path to image
        dst = os.path.join(dst_path, fname)
        cv2.imwrite(dst, clear)

In [ ]:
!rm -rf data/test_segmented
seg_test_dir = os.path.join("data", "test_segmented")
os.mkdir(seg_test_dir)
test = os.path.join("data", "test")
test_list = os.listdir(test)

for folder in test_list:
    
    # create a path to the folder
    path = os.path.join(test, str(folder))
    dst_path = os.path.join(seg_test_dir, str(folder))
    # create a list of all files in the folder
    file_list = os.listdir(path)

    # move the 0 images to all_images_dir
    for fname in file_list:
      
        if not os.path.exists(dst_path):
            os.mkdir(dst_path)

        # source path to image
        src = os.path.join(path, fname)
        img = cv2.imread(src)
        blurImg = cv2.GaussianBlur(img, (5, 5), 0)   
    
        # Convert to HSV image
        hsvImg = cv2.cvtColor(blurImg, cv2.COLOR_BGR2HSV)  

        # Create mask (parameters - green color range)
        lower_green = (25, 40, 50)
        upper_green = (75, 255, 255)
        mask = cv2.inRange(hsvImg, lower_green, upper_green)  
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (11, 11))
        mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)

        # Create bool mask
        bMask = mask > 0  

        # Apply the mask
        clear = np.zeros_like(img, np.uint8)  # Create empty image
        clear[bMask] = img[bMask]
        
        # destination path to image
        dst = os.path.join(dst_path, fname)
        cv2.imwrite(dst, clear)

In [ ]:
!rm -rf data/test_segmented_one
new_test_dir = os.path.join("data", "test_segmented_one")
os.mkdir(new_test_dir)
test = os.path.join("data", "test_segmented")
test_list = os.listdir(test)

for folder in test_list:
    
    # create a path to the folder
    path =os.path.join(test, str(folder))

    # create a list of all files in the folder
    file_list = os.listdir(path)

    # move the 0 images to all_images_dir
    for fname in file_list:

        # source path to image
        src = os.path.join(path, fname)
        
        # Change the file name because many images have the same file name.
        # Add the folder name to the existing file name.
        
        # destination path to image
        dst = os.path.join(new_test_dir, fname)
        # copy the image from the source to the destination
        shutil.copyfile(src, dst)

In [ ]:
!cp trained_model.pkl gdrive/My\ Drive/Colab/.

In [ ]:
!cp gdrive/My\ Drive/Colab/start_350_v5.pth .